In [1]:
!pip install tonic --quiet
!pip install snntorch --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.5/107.5 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.3/125.3 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 75.2 MB/s eta 0:00:00


In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

import torchvision
# import torchvision.transforms as transforms
from tonic import datasets, transforms

import tonic
import snntorch as snn
from snntorch import functional as SF

from tonic import DiskCachedDataset
from torch.utils.data import DataLoader

import numpy as np

import time

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [18]:
# Define the fully connected network

class FCN_1(nn.Module):
    def __init__(self, neurons_in=700, neurons_h1=256, bn=False, dropout=0.0, beta=0.95):
        super(FCN_1, self).__init__()
        self.fc1 = nn.Linear(neurons_in, neurons_h1)
        self.fc_ = nn.Linear(neurons_h1, 20)
        self.lif1 = snn.Leaky(beta=beta)
        self.lif_ = snn.Leaky(beta=beta)
        if dropout > 0:
          self.drop1 = nn.Dropout(p=dropout)
        if bn:
          self.bn1 = nn.BatchNorm1d(neurons_h1)

    def forward(self, x):
      spk_rec = []
      mem_rec = []

      mem1 = self.lif1.init_leaky()
      mem_ = self.lif_.init_leaky()

      for step in range(x.size(0)):
        cur1 = self.fc1(x[step].squeeze(1))
        if hasattr(self, 'drop1'):
          cur1 = self.drop1(cur1)
        if hasattr(self, 'bn1'):
          cur1 = self.bn1(cur1)
        spk1, mem1 = self.lif1(cur1, mem1)

        cur_ = self.fc_(spk1)
        spk_, mem_ = self.lif_(cur_, mem_)
        spk_rec.append(spk_)
        mem_rec.append(mem_)
      return torch.stack(spk_rec), torch.stack(mem_rec)

class FCN_2(nn.Module):
    def __init__(self, neurons_in=700, neurons_h1=256, neurons_h2=256, bn=False, dropout=0.0, beta=0.95):
        super(FCN_2, self).__init__()
        self.fc1 = nn.Linear(neurons_in, neurons_h1)
        self.fc2 = nn.Linear(neurons_h1, neurons_h2)
        self.fc_ = nn.Linear(neurons_h2, 20)
        self.lif1 = snn.Leaky(beta=beta)
        self.lif2 = snn.Leaky(beta=beta)
        self.lif_ = snn.Leaky(beta=beta)
        if dropout > 0:
          self.drop1 = nn.Dropout(p=dropout)
          self.drop2 = nn.Dropout(p=dropout)
        if bn:
          self.bn1 = nn.BatchNorm1d(neurons_h1)
          self.bn2 = nn.BatchNorm1d(neurons_h2)

    def forward(self, x):
      spk_rec = []
      mem_rec = []

      mem1 = self.lif1.init_leaky()
      mem2 = self.lif2.init_leaky()
      mem_ = self.lif_.init_leaky()

      for step in range(x.size(0)):
        cur1 = self.fc1(x[step].squeeze(1))
        if hasattr(self, 'drop1'):
          cur1 = self.drop1(cur1)
        if hasattr(self, 'bn1'):
          cur1 = self.bn1(cur1)
        spk1, mem1 = self.lif1(cur1, mem1)

        cur2 = self.fc2(spk1)
        if hasattr(self, 'drop2'):
          cur2 = self.drop2(cur2)
        if hasattr(self, 'bn2'):
          cur2 = self.bn2(cur2)
        spk2, mem2 = self.lif2(cur2, mem2)

        cur_ = self.fc_(spk2)
        spk_, mem_ = self.lif_(cur_, mem_)
        spk_rec.append(spk_)
        mem_rec.append(mem_)
      return torch.stack(spk_rec), torch.stack(mem_rec)

class FCN_3(nn.Module):
    def __init__(self, neurons_in=700, neurons_h1=256, neurons_h2=256, neurons_h3=256, bn=False, dropout=0.0, beta=0.95):
        super(FCN_3, self).__init__()
        self.fc1 = nn.Linear(neurons_in, neurons_h1)
        self.fc2 = nn.Linear(neurons_h1, neurons_h2)
        self.fc3 = nn.Linear(neurons_h2, neurons_h3)
        self.fc_ = nn.Linear(neurons_h3, 20)
        self.lif1 = snn.Leaky(beta=beta)
        self.lif2 = snn.Leaky(beta=beta)
        self.lif3 = snn.Leaky(beta=beta)
        self.lif_ = snn.Leaky(beta=beta)
        if dropout > 0:
          self.drop1 = nn.Dropout(p=dropout)
          self.drop2 = nn.Dropout(p=dropout)
          self.drop3 = nn.Dropout(p=dropout)
        if bn:
          self.bn1 = nn.BatchNorm1d(neurons_h1)
          self.bn2 = nn.BatchNorm1d(neurons_h2)
          self.bn3 = nn.BatchNorm1d(neurons_h3)

    def forward(self, x):
      spk_rec = []
      mem_rec = []

      mem1 = self.lif1.init_leaky()
      mem2 = self.lif2.init_leaky()
      mem3 = self.lif2.init_leaky()
      mem_ = self.lif_.init_leaky()

      for step in range(x.size(0)):
        cur1 = self.fc1(x[step].squeeze(1))
        if hasattr(self, 'drop1'):
          cur1 = self.drop1(cur1)
        if hasattr(self, 'bn1'):
          cur1 = self.bn1(cur1)
        spk1, mem1 = self.lif1(cur1, mem1)

        cur2 = self.fc2(spk1)
        if hasattr(self, 'drop2'):
          cur2 = self.drop2(cur2)
        if hasattr(self, 'bn2'):
          cur2 = self.bn2(cur2)
        spk2, mem2 = self.lif2(cur2, mem2)

        cur3 = self.fc3(spk2)
        if hasattr(self, 'drop3'):
          cur3 = self.drop3(cur3)
        if hasattr(self, 'bn3'):
          cur3 = self.bn3(cur3)
        spk3, mem3 = self.lif3(cur3, mem3)

        cur_ = self.fc_(spk3)
        spk_, mem_ = self.lif_(cur_, mem_)
        spk_rec.append(spk_)
        mem_rec.append(mem_)
      return torch.stack(spk_rec), torch.stack(mem_rec)


class FCN_4(nn.Module):
    def __init__(self, neurons_in=700, neurons_h1=256, neurons_h2=256, neurons_h3=256, neurons_h4=256, bn=False, dropout=0.0, beta=0.95):
        super(FCN_4, self).__init__()
        self.fc1 = nn.Linear(neurons_in, neurons_h1)
        self.fc2 = nn.Linear(neurons_h1, neurons_h2)
        self.fc3 = nn.Linear(neurons_h3, neurons_h3)
        self.fc4 = nn.Linear(neurons_h3, neurons_h4)
        self.fc_ = nn.Linear(neurons_h4, 20)
        self.lif1 = snn.Leaky(beta=beta)
        self.lif2 = snn.Leaky(beta=beta)
        self.lif3 = snn.Leaky(beta=beta)
        self.lif4 = snn.Leaky(beta=beta)
        self.lif_ = snn.Leaky(beta=beta)
        if dropout > 0:
          self.drop1 = nn.Dropout(p=dropout)
          self.drop2 = nn.Dropout(p=dropout)
          self.drop3 = nn.Dropout(p=dropout)
          self.drop4 = nn.Dropout(p=dropout)
        if bn:
          self.bn1 = nn.BatchNorm1d(neurons_h1)
          self.bn2 = nn.BatchNorm1d(neurons_h2)
          self.bn3 = nn.BatchNorm1d(neurons_h3)
          self.bn4 = nn.BatchNorm1d(neurons_h4)

    def forward(self, x):
      spk_rec = []
      mem_rec = []

      mem1 = self.lif1.init_leaky()
      mem2 = self.lif2.init_leaky()
      mem3 = self.lif2.init_leaky()
      mem4 = self.lif2.init_leaky()
      mem_ = self.lif_.init_leaky()

      for step in range(x.size(0)):
        cur1 = self.fc1(x[step].squeeze(1))
        if hasattr(self, 'drop1'):
          cur1 = self.drop1(cur1)
        if hasattr(self, 'bn1'):
          cur1 = self.bn1(cur1)
        spk1, mem1 = self.lif1(cur1, mem1)

        cur2 = self.fc2(spk1)
        if hasattr(self, 'drop2'):
          cur2 = self.drop2(cur2)
        if hasattr(self, 'bn2'):
          cur2 = self.bn2(cur2)
        spk2, mem2 = self.lif2(cur2, mem2)

        cur3 = self.fc3(spk2)
        if hasattr(self, 'drop3'):
          cur3 = self.drop3(cur3)
        if hasattr(self, 'bn3'):
          cur3 = self.bn3(cur3)
        spk3, mem3 = self.lif3(cur3, mem3)

        cur4 = self.fc4(spk3)
        if hasattr(self, 'drop4'):
          cur4 = self.drop4(cur4)
        if hasattr(self, 'bn4'):
          cur4 = self.bn4(cur4)
        spk4, mem4 = self.lif4(cur4, mem4)

        cur_ = self.fc_(spk4)
        spk_, mem_ = self.lif_(cur_, mem_)
        spk_rec.append(spk_)
        mem_rec.append(mem_)
      return torch.stack(spk_rec), torch.stack(mem_rec)


In [52]:
def train(model, criterion, optimizer, trainloader, device, num_epochs=30, testloader=None, scheduler=None):
  t = time.process_time()
  for epoch in range(num_epochs):
      # Training loop
      model.train()
      for batch_idx, (data, target) in enumerate(trainloader):
          optimizer.zero_grad()
          spk_rec, mem_rec = model(data.to(device))
          # spk_rec = model(data.to(device))[-1]
          loss = criterion(spk_rec, target.to(device))
          loss.backward()
          optimizer.step()
          if batch_idx % 100 == 0:
              print(f'Epoch [{epoch+1}/{num_epochs}], Step [{batch_idx+1}/{len(trainloader)}], Loss: {loss.item():.4f}')
      # if scheduler is not None:
      #     model.eval()
      #     correct = 0
      #     total = 0
      #     for data, target in testloader:
      #         spk_rec, mem_rec = model(data.to(device))
      #         correct += SF.accuracy_rate(spk_rec, target.to(device)) * target.size(0)
      #         total += spk_rec.size(1)
      #     scheduler.step(correct / total)
      if scheduler is not None:
          scheduler.step()
      if epoch % 10 == 0 and testloader is not None:
          accuracy(model, testloader, device)
  print(f'Training time: {time.process_time() - t}')


def accuracy(model, testloader, device):
    # Testing loop
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        correct2 = 0
        total2 = 0
        for data, target in testloader:
            spk_rec, mem_rec = model(data.to(device))
            # spk_rec = model(data.to(device))[-1]
            # print('input: ', data.size(), ' output: ', spk_rec.size())
            correct += SF.accuracy_rate(spk_rec, target.to(device)) * target.size(0)
            total += spk_rec.size(1)
            total2 += target.size(0)
            # print(correct, ' out of ', target.size())

            # correct2 += (predicted == target).sum().item()

        print(f'Accuracy of the network on test set: {100 * correct / total:.2f}% ({100 * correct / total2:.2f}%)')

def accuracy_2class(model, testloader, device):
    # Testing loop
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for data, target in testloader:
            spk_rec, mem_rec = model(data.to(device))
            _, idx = spk_rec.sum(dim=0).max(1)
            idx = (idx < 10)
            target = (target.to(device) < 10)
            equal = (target.to(device) == idx.to(device)).detach().cpu().numpy()
            correct += np.mean(equal) * target.size(0)
            total += target.size(0)
        print(f'Accuracy of the network on 2 classes: {100 * correct / total:.2f}%')

In [56]:
batch_size = 256 # gives the best tradeoff between speed (~10 sec per epoch on T4 GPU) and number of weight updates (32 steps per 10 sec)
dt = 62_500/2
transform = transforms.Compose(
            [
                transforms.CropTime(max=1e6), # all timestamp units in microseconds in Tonic
                transforms.Downsample(spatial_factor=512.0/700.0),
                transforms.ToFrame(
                    sensor_size=(512,1,1),
                    time_window=dt,
                    include_incomplete=True,
                ),
            ]
        )
trainset=datasets.SHD('data', transform=transform)
testset=datasets.SHD('data', transform=transform, train=False)
shd_trainset = DiskCachedDataset(trainset, cache_path='./cache/shd/train/512_'+str(dt)+'/')
shd_testset = DiskCachedDataset(testset, cache_path='./cache/shd/test/512_'+str(dt)+'/')
trainloader = DataLoader(shd_trainset, batch_size=batch_size, collate_fn=tonic.collation.PadTensors(batch_first=False), shuffle=True)
testloader = DataLoader(shd_testset, batch_size=batch_size, collate_fn=tonic.collation.PadTensors(batch_first=False), shuffle=True)

for data, labels in iter(trainloader):
  print(data.size(), labels.size(), labels)
  break

learning_rate = 0.001
num_epochs = 150

# Model, loss function, and optimizer
model = FCN_1(neurons_in=512, neurons_h1=256, bn=False).to(device)
criterion = SF.ce_rate_loss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = lr_scheduler.LinearLR(optimizer, start_factor=1.0, end_factor=0.1, total_iters=100)

t = time.process_time()
train(model, criterion, optimizer, trainloader, device, num_epochs,    testloader, scheduler=scheduler)
print(time.process_time() - t)
accuracy(model, testloader, device)
accuracy_2class(model, testloader, device)

torch.Size([32, 256, 1, 512]) torch.Size([256]) tensor([19,  7, 15, 14,  3, 14,  7, 11,  0, 19, 17,  7,  9,  0,  8,  0, 16, 17,
        15,  5, 15, 10,  1,  9, 11, 19, 16,  4, 19, 18, 16,  6,  5, 18, 15, 13,
        12, 11, 19,  5, 16,  3,  8,  3,  2, 14, 13, 14,  2,  0, 10, 19, 16,  0,
         7,  6,  3,  3, 11,  4, 19,  5,  6,  0, 17, 15,  9,  1,  6, 19, 15,  6,
        14,  7,  0, 10,  5,  6,  2, 17, 17, 15, 11,  7, 15,  9,  5, 14,  1, 16,
         5,  5, 15,  2, 19, 13,  3, 12,  6, 10,  3, 17,  6,  8, 17,  6, 15,  7,
        17, 11,  6,  3,  4,  1, 19, 14,  4,  3, 19, 10,  9,  7, 10, 17, 12, 17,
         6, 11,  3, 17,  1, 13,  8, 11,  2, 11, 11,  2,  7, 15, 17,  6, 16, 18,
         6, 17,  2,  4, 14, 17,  0, 16, 12, 14,  1,  0,  1,  4,  7,  0,  3, 17,
         4,  1,  0,  5, 19,  6,  7, 19,  2, 11,  5, 19,  4,  2, 15, 19,  5,  2,
        17, 15,  5, 11, 13,  4,  6,  1,  7, 16, 10, 14,  1,  1,  4,  3,  3,  1,
        18,  5, 19, 16, 16,  0,  2, 13,  6,  6,  4,  2, 14, 13,  6, 14, 

In [13]:
batch_size = 256 # gives the best tradeoff between speed (~10 sec per epoch on T4 GPU) and number of weight updates (32 steps per 10 sec)
dt = 62_500/2 #dt = 125_000
transform = transforms.Compose(
            [
                transforms.CropTime(max=1e6), # all timestamp units in microseconds in Tonic
                transforms.Downsample(spatial_factor=512.0/700.0),
                transforms.ToFrame(
                    sensor_size=(512,1,1),
                    time_window=dt,
                    include_incomplete=True,
                ),
            ]
        )
trainset=datasets.SHD('data', transform=transform)
testset=datasets.SHD('data', transform=transform, train=False)
shd_trainset = DiskCachedDataset(trainset, cache_path='./cache/shd/train/512_'+str(dt)+'/')
shd_testset = DiskCachedDataset(testset, cache_path='./cache/shd/test/512_'+str(dt)+'/')
trainloader = DataLoader(shd_trainset, batch_size=batch_size, collate_fn=tonic.collation.PadTensors(batch_first=False), shuffle=True)
testloader = DataLoader(shd_testset, batch_size=batch_size, collate_fn=tonic.collation.PadTensors(batch_first=False), shuffle=True)

for data, labels in iter(trainloader):
  print(data.size(), labels.size(), labels)
  break

learning_rate = 0.001
num_epochs = 3

# Model, loss function, and optimizer
model = FCN_1(neurons_in=512, neurons_h1=256, bn=False).to(device)
criterion = SF.ce_rate_loss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = lr_scheduler.LinearLR(optimizer, start_factor=1.0, end_factor=0.1, total_iters=100)

import time
t = time.process_time()
train(model, criterion, optimizer, trainloader, device, num_epochs,    testloader, scheduler=scheduler)
print(time.process_time() - t)
accuracy(model, testloader, device)

torch.Size([32, 256, 1, 512]) torch.Size([256]) tensor([17, 13, 19, 16,  8, 12,  4, 10, 18,  0,  4, 15,  1, 16, 14,  0,  0, 13,
        19, 10, 18, 16,  1, 13,  5,  1, 11, 17, 15,  2, 10,  8,  1,  7, 12, 16,
         7,  9,  7,  9, 13, 13,  8,  6,  2, 11, 19, 19, 17, 14,  2, 14, 17,  5,
         8,  6, 10, 15,  7, 15,  2,  2,  5, 12, 12,  1, 10,  4,  6, 13,  5, 17,
        19,  7, 19,  1, 14, 19,  7, 14, 16,  8, 15, 14,  6, 12,  3,  5, 10,  5,
         6,  9, 16,  7,  1, 11,  0, 18, 18, 10, 11,  2, 16,  9,  9,  0,  3, 13,
        12,  5,  6,  5,  1,  5, 12, 12,  7, 14, 15,  8, 19, 19, 13,  5,  1, 12,
         5, 15,  6,  6,  3,  0,  7, 18, 11,  3, 11, 15, 13, 17,  8, 15,  8, 13,
         5,  7,  4,  6, 17,  5, 13,  2,  0, 15,  0, 13,  4, 19, 10, 17, 10, 18,
         2,  7, 11,  6, 11, 13, 12, 12, 14,  4,  5, 14,  5,  9,  1,  6, 14, 12,
         7, 19,  3, 17,  4,  0,  2, 14,  5, 12, 16,  8,  2,  0,  7, 15, 15, 15,
         4, 12, 15,  9,  8,  6,  1, 19, 10,  7, 18,  9, 13,  8, 14,  0, 

In [11]:
batch_size = 256 # gives the best tradeoff between speed (~10 sec per epoch on T4 GPU) and number of weight updates (32 steps per 10 sec)
dt = 62_500/4 #dt = 125_000
transform = transforms.Compose(
            [
                transforms.CropTime(max=1e6), # all timestamp units in microseconds in Tonic
                transforms.Downsample(spatial_factor=512.0/700.0),
                transforms.ToFrame(
                    sensor_size=(512,1,1),
                    time_window=dt,
                    include_incomplete=True,
                ),
            ]
        )
trainset=datasets.SHD('data', transform=transform)
testset=datasets.SHD('data', transform=transform, train=False)
shd_trainset = DiskCachedDataset(trainset, cache_path='./cache/shd/train/512_'+str(dt)+'/')
shd_testset = DiskCachedDataset(testset, cache_path='./cache/shd/test/512_'+str(dt)+'/')
trainloader = DataLoader(shd_trainset, batch_size=batch_size, collate_fn=tonic.collation.PadTensors(batch_first=False), shuffle=True)
testloader = DataLoader(shd_testset, batch_size=batch_size, collate_fn=tonic.collation.PadTensors(batch_first=False), shuffle=True)

for data, labels in iter(trainloader):
  print(data.size(), labels.size(), labels)
  break

learning_rate = 0.001
num_epochs = 3

# Model, loss function, and optimizer
model = FCN_1(neurons_in=512, neurons_h1=256, bn=False).to(device)
criterion = SF.ce_rate_loss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = lr_scheduler.LinearLR(optimizer, start_factor=1.0, end_factor=0.1, total_iters=100)

import time
t = time.process_time()
train(model, criterion, optimizer, trainloader, device, num_epochs,    testloader, scheduler=scheduler)
print(time.process_time() - t)
accuracy(model, testloader, device)

torch.Size([64, 256, 1, 512]) torch.Size([256]) tensor([19,  0,  9, 14, 12, 12,  8, 12, 16,  4, 13,  7,  3,  3,  0, 16,  3, 11,
         7,  5, 17, 18,  7,  4,  3,  1,  4, 15,  3,  7,  8, 18, 16, 10,  4, 10,
        14, 18, 14, 17,  1,  0, 17,  2, 17, 14, 10,  1, 16,  7,  4,  4,  3,  1,
        15,  3, 18,  8, 10, 12, 14,  6,  4,  6,  6,  3, 10, 17, 17,  6, 11, 17,
         3,  3, 12,  8,  9,  6, 19,  2,  1, 11,  5, 15, 18,  1, 16, 17, 10, 15,
        15,  8, 10, 17,  5,  0,  2, 13, 15, 15,  1,  3, 15,  2,  9, 11, 13, 12,
        16, 15,  1,  9,  5,  1,  6,  8, 19, 11,  1, 14,  6,  7,  4, 14,  8, 16,
         5, 19, 13,  6, 11, 11, 14, 19, 15, 12, 10,  8,  2,  8,  3,  4,  5,  8,
        10,  9, 11, 14,  2,  0, 17,  9, 11, 10, 13, 13, 18,  9, 15,  7, 12, 12,
         9,  5,  2, 15,  8, 13,  6,  0,  7,  0,  9,  3, 15, 11,  1,  4,  4,  9,
        10, 15, 17,  5,  5,  7, 13,  9, 19,  0, 15, 14, 16,  6, 19,  0, 19, 10,
        10,  9,  3,  8,  1, 15, 14, 12, 13, 17, 16, 17, 10,  1,  1, 13, 

Deeper networks

In [55]:
batch_size = 256 # gives the best tradeoff between speed (~10 sec per epoch on T4 GPU) and number of weight updates (32 steps per 10 sec)
dt = 62_500/2 #dt = 125_000
transform = transforms.Compose(
            [
                transforms.CropTime(max=1e6), # all timestamp units in microseconds in Tonic
                transforms.Downsample(spatial_factor=512.0/700.0),
                transforms.ToFrame(
                    sensor_size=(512,1,1),
                    time_window=dt,
                    include_incomplete=True,
                ),
            ]
        )
trainset=datasets.SHD('data', transform=transform)
testset=datasets.SHD('data', transform=transform, train=False)
shd_trainset = DiskCachedDataset(trainset, cache_path='./cache/shd/train/512_'+str(dt)+'/')
shd_testset = DiskCachedDataset(testset, cache_path='./cache/shd/test/512_'+str(dt)+'/')
trainloader = DataLoader(shd_trainset, batch_size=batch_size, collate_fn=tonic.collation.PadTensors(batch_first=False), shuffle=True)
testloader = DataLoader(shd_testset, batch_size=batch_size, collate_fn=tonic.collation.PadTensors(batch_first=False), shuffle=True)

for data, labels in iter(trainloader):
  print(data.size(), labels.size(), labels)
  break

learning_rate = 0.001
num_epochs = 3

# Model, loss function, and optimizer
model = FCN_3(neurons_in=512, neurons_h1=256, neurons_h2=256, neurons_h3=256, dropout=0.5, bn=False).to(device)
criterion = SF.ce_rate_loss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = lr_scheduler.LinearLR(optimizer, start_factor=1.0, end_factor=0.1, total_iters=100)

import time
t = time.process_time()
train(model, criterion, optimizer, trainloader, device, num_epochs,    testloader, scheduler=scheduler)
print(time.process_time() - t)
accuracy(model, testloader, device)

torch.Size([32, 256, 1, 512]) torch.Size([256]) tensor([17,  0, 16,  9,  3,  9,  9,  4,  2, 12,  3, 14,  8, 18,  9, 14,  4,  9,
        17,  5,  9, 19,  6,  9,  7, 16, 11,  1, 18,  1, 19,  9,  0, 15, 13, 18,
         6, 11, 16, 11,  6, 16,  8, 10,  0, 18, 18, 15,  7, 12,  6,  6, 16,  3,
        18, 18,  2,  0,  5,  5,  3, 12,  3,  4,  2,  9,  9, 12,  9, 13,  7, 15,
         3, 10,  4,  9, 16, 17, 15,  4,  3, 15,  4,  3, 15, 15,  1,  4, 12, 11,
         0, 15, 17,  2,  8,  8,  8, 19,  8, 13, 16,  5,  1, 12,  5,  9, 12, 10,
        13,  2,  2,  7, 10, 16,  7, 19,  1, 18,  7, 16,  0, 14, 18,  9,  0,  3,
        16, 11, 16,  9,  6, 17,  0, 10,  9,  6, 14,  3,  8,  6,  6,  5,  5,  9,
        19,  8, 19, 17, 18, 15,  5, 13,  7, 16, 14,  5,  4,  1,  2,  7,  0,  4,
        15,  2,  7,  5,  0, 12,  3,  2, 15,  3,  3,  1, 12,  5,  4, 18,  0, 14,
        15, 11, 19, 19,  2, 15,  6,  6,  4, 15, 15, 17,  6, 12, 13, 12, 11, 13,
        16, 15,  0,  1, 19,  4,  5, 15,  4, 16,  7, 18, 17,  6, 12,  1, 

In [16]:
batch_size = 256 # gives the best tradeoff between speed (~10 sec per epoch on T4 GPU) and number of weight updates (32 steps per 10 sec)
dt = 62_500/2 #dt = 125_000
transform = transforms.Compose(
            [
                transforms.CropTime(max=1e6), # all timestamp units in microseconds in Tonic
                transforms.Downsample(spatial_factor=512.0/700.0),
                transforms.ToFrame(
                    sensor_size=(512,1,1),
                    time_window=dt,
                    include_incomplete=True,
                ),
            ]
        )
trainset=datasets.SHD('data', transform=transform)
testset=datasets.SHD('data', transform=transform, train=False)
shd_trainset = DiskCachedDataset(trainset, cache_path='./cache/shd/train/512_'+str(dt)+'/')
shd_testset = DiskCachedDataset(testset, cache_path='./cache/shd/test/512_'+str(dt)+'/')
trainloader = DataLoader(shd_trainset, batch_size=batch_size, collate_fn=tonic.collation.PadTensors(batch_first=False), shuffle=True)
testloader = DataLoader(shd_testset, batch_size=batch_size, collate_fn=tonic.collation.PadTensors(batch_first=False), shuffle=True)

for data, labels in iter(trainloader):
  print(data.size(), labels.size(), labels)
  break

learning_rate = 0.001
num_epochs = 3

# Model, loss function, and optimizer
model = FCN_4(neurons_in=512, neurons_h1=256, neurons_h2=256, neurons_h3=256, neurons_h4=256, dropout=0.5, bn=False).to(device)
criterion = SF.ce_rate_loss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = lr_scheduler.LinearLR(optimizer, start_factor=1.0, end_factor=0.1, total_iters=100)


train(model, criterion, optimizer, trainloader, device, num_epochs,    testloader, scheduler=scheduler)
accuracy(model, testloader, device)

torch.Size([32, 256, 1, 512]) torch.Size([256]) tensor([11, 17, 16,  8, 19,  5, 14,  5, 14, 17, 13,  2, 18,  1, 14,  9, 19,  2,
         2, 14, 14, 14,  3, 13, 18, 13,  3,  0,  8, 13,  3, 16, 10,  6,  8, 16,
        13,  9,  4, 12, 13,  1, 16, 14, 13, 16, 16, 13, 17,  0, 11,  3, 19, 19,
        13,  7, 19, 18,  5,  0,  8, 12, 15, 10,  2,  7,  2, 19, 13, 15, 12,  4,
         8,  6, 11,  8,  6,  6,  0,  6, 16,  0, 16,  8,  9, 16,  8, 16,  5,  7,
         5,  0,  4, 12,  6,  8, 17, 13, 15,  5,  8,  6, 19,  2,  7, 15,  1,  8,
         9, 10,  5, 13, 11, 11,  6, 15, 19,  2, 18, 15, 19, 12, 12, 14,  9,  6,
        14, 11, 11, 12, 18, 15, 18, 18,  4, 13, 16,  6, 12,  0, 12, 14, 17,  9,
        12,  9, 16,  9, 17,  3,  0, 16, 15, 19,  2,  6, 15, 11,  4, 18,  4, 18,
        11,  2,  0,  6,  7, 18,  8,  2, 12,  0, 13,  8, 13,  1, 15, 10,  4,  0,
        10, 12,  5,  4, 16,  9,  9,  1, 15, 17,  8, 17, 11, 14, 15,  4, 14, 11,
        11, 12,  7, 15,  9, 18,  0,  7,  4,  9,  4,  2,  4,  8, 10, 14, 

In [19]:
batch_size = 256 # gives the best tradeoff between speed (~10 sec per epoch on T4 GPU) and number of weight updates (32 steps per 10 sec)
dt = 62_500/2 #dt = 125_000
transform = transforms.Compose(
            [
                transforms.CropTime(max=1e6), # all timestamp units in microseconds in Tonic
                transforms.Downsample(spatial_factor=512.0/700.0),
                transforms.ToFrame(
                    sensor_size=(512,1,1),
                    time_window=dt,
                    include_incomplete=True,
                ),
            ]
        )
trainset=datasets.SHD('data', transform=transform)
testset=datasets.SHD('data', transform=transform, train=False)
shd_trainset = DiskCachedDataset(trainset, cache_path='./cache/shd/train/512_'+str(dt)+'/')
shd_testset = DiskCachedDataset(testset, cache_path='./cache/shd/test/512_'+str(dt)+'/')
trainloader = DataLoader(shd_trainset, batch_size=batch_size, collate_fn=tonic.collation.PadTensors(batch_first=False), shuffle=True)
testloader = DataLoader(shd_testset, batch_size=batch_size, collate_fn=tonic.collation.PadTensors(batch_first=False), shuffle=True)

for data, labels in iter(trainloader):
  print(data.size(), labels.size(), labels)
  break

learning_rate = 0.001
num_epochs = 3

# Model, loss function, and optimizer
model = FCN_4(neurons_in=512, neurons_h1=256, neurons_h2=256, neurons_h3=256, neurons_h4=256, dropout=0.5, bn=True).to(device)
criterion = SF.ce_rate_loss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = lr_scheduler.LinearLR(optimizer, start_factor=1.0, end_factor=0.1, total_iters=100)

import time
t = time.process_time()
train(model, criterion, optimizer, trainloader, device, num_epochs,    testloader, scheduler=scheduler)
print(time.process_time() - t)
accuracy(model, testloader, device)

torch.Size([32, 256, 1, 512]) torch.Size([256]) tensor([18, 15, 13, 15, 10,  9,  2,  5, 16, 17,  3,  6, 19, 12, 13, 17, 14, 19,
        14, 18, 16,  5, 14, 13, 12,  5, 18,  2, 19,  4,  5,  9, 14,  2,  7, 10,
        10,  9, 14,  3,  7, 19,  6,  6, 14, 10, 18, 14,  8, 15, 19, 11,  5, 13,
         5, 15,  5,  6, 18,  6, 17, 13,  1,  4,  0,  2,  2, 10, 16, 12, 15, 11,
         0,  5, 12, 16,  1,  2,  2,  1, 14,  9,  3, 11, 15,  5,  3, 16, 12, 13,
        11,  9,  4, 18,  1, 16,  5,  8,  8,  6,  6, 14, 14, 12,  2,  4, 19, 10,
         4,  6,  7,  8, 10,  9, 12, 11, 18, 11, 19, 18,  7, 19, 17, 13, 17, 11,
         6,  6, 11, 16, 14, 16, 10,  9,  1, 12, 17, 16, 13, 19,  4, 13, 10, 13,
        19, 11, 12,  9,  2, 10, 16,  7,  4, 16, 15,  5, 15, 13,  8, 16,  6, 12,
         6,  9, 11,  6, 19, 16,  9, 18, 10, 11, 13, 14, 11,  0, 18,  8,  4, 16,
         0, 12, 14,  3, 16,  9,  5,  7,  7, 13, 11,  2, 16,  3,  2,  5,  5, 18,
         5, 13, 13,  0, 12, 15, 15, 13,  1, 10,  2, 19,  5,  8,  8,  2, 

In [20]:
batch_size = 256 # gives the best tradeoff between speed (~10 sec per epoch on T4 GPU) and number of weight updates (32 steps per 10 sec)
dt = 62_500/4 #dt = 125_000
transform = transforms.Compose(
            [
                transforms.CropTime(max=1e6), # all timestamp units in microseconds in Tonic
                transforms.Downsample(spatial_factor=512.0/700.0),
                transforms.ToFrame(
                    sensor_size=(512,1,1),
                    time_window=dt,
                    include_incomplete=True,
                ),
            ]
        )
trainset=datasets.SHD('data', transform=transform)
testset=datasets.SHD('data', transform=transform, train=False)
shd_trainset = DiskCachedDataset(trainset, cache_path='./cache/shd/train/512_'+str(dt)+'/')
shd_testset = DiskCachedDataset(testset, cache_path='./cache/shd/test/512_'+str(dt)+'/')
trainloader = DataLoader(shd_trainset, batch_size=batch_size, collate_fn=tonic.collation.PadTensors(batch_first=False), shuffle=True)
testloader = DataLoader(shd_testset, batch_size=batch_size, collate_fn=tonic.collation.PadTensors(batch_first=False), shuffle=True)

for data, labels in iter(trainloader):
  print(data.size(), labels.size(), labels)
  break

learning_rate = 0.001
num_epochs = 200 # 67% after 100, 69% after 200

# Model, loss function, and optimizer
model = FCN_4(neurons_in=512, neurons_h1=256, neurons_h2=256, neurons_h3=256, neurons_h4=256, dropout=0.5, bn=False).to(device)
criterion = SF.ce_rate_loss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = lr_scheduler.LinearLR(optimizer, start_factor=1.0, end_factor=0.1, total_iters=100)

import time
t = time.process_time()
train(model, criterion, optimizer, trainloader, device, num_epochs,    testloader, scheduler=scheduler)
print(time.process_time() - t)
accuracy(model, testloader, device)

torch.Size([64, 256, 1, 512]) torch.Size([256]) tensor([ 7,  9,  5, 10,  0,  2, 11, 12,  9,  2,  3, 13,  8, 12,  5, 12, 16, 16,
        19, 16, 16,  4,  2,  8,  0, 14, 18, 10,  7,  0, 14,  7, 11, 19,  8,  8,
        17, 18, 10, 13, 10, 18, 18,  8, 14,  0, 11,  3,  2,  8, 19,  2, 12,  0,
         2,  1,  2, 17,  4,  5,  5,  1,  7,  7,  4,  7,  1, 18, 10,  1,  9, 17,
        11,  1,  8,  7, 11, 13,  5, 17, 13, 13, 14,  0,  2,  8,  5, 10,  4, 12,
        11, 17, 14, 13, 15,  4, 16,  4, 13, 19, 11,  7, 16, 19, 12, 14, 16,  6,
         7,  5, 10, 10,  7,  4, 11, 14, 17,  7,  3,  2, 18,  5, 13,  8,  6,  1,
         2, 17, 14, 11,  4,  1, 19, 10, 19,  0, 14,  0, 17,  9,  0, 10,  6,  3,
        14, 15,  6, 18, 12,  9,  1, 13, 10, 18,  7,  3, 17,  1,  1,  1, 17, 17,
        12,  4,  9, 12,  0,  8,  2,  7, 16,  0,  9,  1,  3,  6,  4, 19, 15, 18,
         3, 17,  7, 15, 18,  0,  5,  0, 16,  1,  6, 17, 19, 17,  0,  2, 19, 18,
        16,  6,  9, 16, 15, 15,  2,  8,  0,  9,  1, 11, 17, 13, 16,  3, 

In [25]:
batch_size = 4096

trainloader = DataLoader(shd_trainset, batch_size=batch_size, collate_fn=tonic.collation.PadTensors(batch_first=False), shuffle=True)
testloader = DataLoader(shd_testset, batch_size=batch_size, collate_fn=tonic.collation.PadTensors(batch_first=False), shuffle=True)

for data, labels in iter(trainloader):
  print(data.size(), labels.size(), labels)
  break

learning_rate = 0.0005
num_epochs = 30

optimizer = optim.Adam(model.parameters(), lr=learning_rate)

t = time.process_time()
train(model, criterion, optimizer, trainloader, device, num_epochs,    testloader)
print(time.process_time() - t)
accuracy(model, testloader, device)

torch.Size([64, 4096, 1, 512]) torch.Size([4096]) tensor([13, 14, 18,  ..., 16, 10,  8])
Epoch [1/30], Step [1/2], Loss: 2.3181
Accuracy of the network on test set: 68.11% (68.11%)
Epoch [2/30], Step [1/2], Loss: 2.3280
Epoch [3/30], Step [1/2], Loss: 2.3300
Epoch [4/30], Step [1/2], Loss: 2.3263
Epoch [5/30], Step [1/2], Loss: 2.3273
Epoch [6/30], Step [1/2], Loss: 2.3242
Epoch [7/30], Step [1/2], Loss: 2.3199
Epoch [8/30], Step [1/2], Loss: 2.3223
Epoch [9/30], Step [1/2], Loss: 2.3196
Epoch [10/30], Step [1/2], Loss: 2.3221
Epoch [11/30], Step [1/2], Loss: 2.3213
Accuracy of the network on test set: 69.13% (69.13%)
Epoch [12/30], Step [1/2], Loss: 2.3165
Epoch [13/30], Step [1/2], Loss: 2.3161
Epoch [14/30], Step [1/2], Loss: 2.3178
Epoch [15/30], Step [1/2], Loss: 2.3192
Epoch [16/30], Step [1/2], Loss: 2.3154
Epoch [17/30], Step [1/2], Loss: 2.3148
Epoch [18/30], Step [1/2], Loss: 2.3105
Epoch [19/30], Step [1/2], Loss: 2.3126
Epoch [20/30], Step [1/2], Loss: 2.3121
Epoch [21/30],

In [27]:
learning_rate = 0.0001
num_epochs = 30

optimizer = optim.Adam(model.parameters(), lr=learning_rate)

train(model, criterion, optimizer, trainloader, device, num_epochs,    testloader)
accuracy(model, testloader, device)

Epoch [1/30], Step [1/2], Loss: 2.3121
Accuracy of the network on test set: 69.26% (69.26%)
Epoch [2/30], Step [1/2], Loss: 2.3119
Epoch [3/30], Step [1/2], Loss: 2.3141
Epoch [4/30], Step [1/2], Loss: 2.3163
Epoch [5/30], Step [1/2], Loss: 2.3135
Epoch [6/30], Step [1/2], Loss: 2.3152
Epoch [7/30], Step [1/2], Loss: 2.3129
Epoch [8/30], Step [1/2], Loss: 2.3147
Epoch [9/30], Step [1/2], Loss: 2.3137
Epoch [10/30], Step [1/2], Loss: 2.3107
Epoch [11/30], Step [1/2], Loss: 2.3122
Accuracy of the network on test set: 69.17% (69.17%)
Epoch [12/30], Step [1/2], Loss: 2.3130
Epoch [13/30], Step [1/2], Loss: 2.3108
Epoch [14/30], Step [1/2], Loss: 2.3130
Epoch [15/30], Step [1/2], Loss: 2.3119
Epoch [16/30], Step [1/2], Loss: 2.3112
Epoch [17/30], Step [1/2], Loss: 2.3132
Epoch [18/30], Step [1/2], Loss: 2.3141
Epoch [19/30], Step [1/2], Loss: 2.3100
Epoch [20/30], Step [1/2], Loss: 2.3129
Epoch [21/30], Step [1/2], Loss: 2.3139
Accuracy of the network on test set: 68.82% (68.82%)
Epoch [22/

In [53]:
accuracy(model, testloader, device)
accuracy_2class(model, testloader, device)

Accuracy of the network on test set: 68.51% (68.51%)
Accuracy of the network on 2 classes: 81.54%


In [57]:
batch_size = 256 # gives the best tradeoff between speed (~10 sec per epoch on T4 GPU) and number of weight updates (32 steps per 10 sec)
dt = 62_500/4
transform = transforms.Compose(
            [
                transforms.CropTime(max=1e6), # all timestamp units in microseconds in Tonic
                transforms.Downsample(spatial_factor=512.0/700.0),
                transforms.ToFrame(
                    sensor_size=(512,1,1),
                    time_window=dt,
                    include_incomplete=True,
                ),
            ]
        )
trainset=datasets.SHD('data', transform=transform)
testset=datasets.SHD('data', transform=transform, train=False)
shd_trainset = DiskCachedDataset(trainset, cache_path='./cache/shd/train/512_'+str(dt)+'/')
shd_testset = DiskCachedDataset(testset, cache_path='./cache/shd/test/512_'+str(dt)+'/')
trainloader = DataLoader(shd_trainset, batch_size=batch_size, collate_fn=tonic.collation.PadTensors(batch_first=False), shuffle=True)
testloader = DataLoader(shd_testset, batch_size=batch_size, collate_fn=tonic.collation.PadTensors(batch_first=False), shuffle=True)

for data, labels in iter(trainloader):
  print(data.size(), labels.size(), labels)
  break

learning_rate = 0.001
num_epochs = 150

# Model, loss function, and optimizer
model = FCN_2(neurons_in=512, neurons_h1=256, neurons_h2=128, bn=False).to(device)
criterion = SF.ce_rate_loss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = lr_scheduler.LinearLR(optimizer, start_factor=1.0, end_factor=0.1, total_iters=100)

t = time.process_time()
train(model, criterion, optimizer, trainloader, device, num_epochs,    testloader, scheduler=scheduler)
print(time.process_time() - t)
accuracy(model, testloader, device)
accuracy_2class(model, testloader, device)

torch.Size([64, 256, 1, 512]) torch.Size([256]) tensor([ 8, 14,  0, 15,  0, 12, 12, 10,  1, 19, 12,  6,  3,  9,  0,  5, 15, 13,
        11,  5, 13, 10, 12, 12, 14,  7, 14,  0,  0,  7, 16,  7,  5,  1, 17, 17,
         8, 11, 14,  1,  6, 15, 10,  7, 11, 18,  4,  5,  7,  0, 19, 18,  0, 19,
        10, 18,  6,  8, 13, 11, 13,  5,  1,  8, 11, 10, 17, 11,  2,  1,  8, 14,
        19, 18, 18, 19, 18,  9, 18,  9, 14, 13,  0,  3, 18,  3, 14, 14,  0, 13,
         0, 13,  4, 13, 16, 13,  7, 14,  8,  6, 16, 18, 14,  5,  6, 17, 11, 16,
        11,  0, 10, 18, 12, 12,  6,  7,  6,  4,  2, 10, 13,  6, 14, 18, 16, 15,
        19,  7,  6, 15, 11,  1,  5,  5, 17, 16, 15, 18, 19,  0, 17, 19,  7, 14,
         1,  9,  9, 18, 13, 14, 13,  8, 11,  7,  3,  5, 14,  9,  9, 15,  8,  2,
         7,  9, 17,  2, 15,  0, 15, 18, 17,  9,  9, 13,  2,  0,  5,  4,  5,  3,
         4,  0,  0, 10,  3,  5, 12, 12, 19, 19, 18,  0,  1, 18,  1,  1,  8,  6,
         6, 17, 11, 18, 17,  2,  5,  7,  1, 17,  1,  6,  3,  4,  7, 10, 

In [60]:
batch_size = 256 # gives the best tradeoff between speed (~10 sec per epoch on T4 GPU) and number of weight updates (32 steps per 10 sec)
dt = 62_500/4
transform = transforms.Compose(
            [
                transforms.CropTime(max=1e6), # all timestamp units in microseconds in Tonic
                transforms.Downsample(spatial_factor=512.0/700.0),
                transforms.ToFrame(
                    sensor_size=(512,1,1),
                    time_window=dt,
                    include_incomplete=True,
                ),
            ]
        )
trainset=datasets.SHD('data', transform=transform)
testset=datasets.SHD('data', transform=transform, train=False)
shd_trainset = DiskCachedDataset(trainset, cache_path='./cache/shd/train/512_'+str(dt)+'/')
shd_testset = DiskCachedDataset(testset, cache_path='./cache/shd/test/512_'+str(dt)+'/')
trainloader = DataLoader(shd_trainset, batch_size=batch_size, collate_fn=tonic.collation.PadTensors(batch_first=False), shuffle=True)
testloader = DataLoader(shd_testset, batch_size=batch_size, collate_fn=tonic.collation.PadTensors(batch_first=False), shuffle=True)

for data, labels in iter(trainloader):
  print(data.size(), labels.size(), labels)
  break

learning_rate = 0.001
num_epochs = 150

# Model, loss function, and optimizer
model = FCN_2(neurons_in=512, neurons_h1=256, neurons_h2=128, dropout=0.5, bn=False).to(device)
criterion = SF.ce_rate_loss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = lr_scheduler.LinearLR(optimizer, start_factor=1.0, end_factor=0.1, total_iters=100)

t = time.process_time()
train(model, criterion, optimizer, trainloader, device, num_epochs,    testloader, scheduler=scheduler)
print(time.process_time() - t)
accuracy(model, testloader, device)
accuracy_2class(model, testloader, device)

torch.Size([64, 256, 1, 512]) torch.Size([256]) tensor([ 2, 10, 18,  2, 16,  6,  9, 17, 12, 12, 13, 13,  5,  8, 19, 19,  8,  6,
        12,  2, 17,  2, 14,  0,  7,  3, 10,  1,  1, 18, 12,  0, 15,  1,  6, 16,
        11,  1, 14, 12,  9, 12, 15,  6,  4,  2,  1,  6,  2, 15, 14, 14,  4,  7,
        19,  9, 15,  6, 11,  0, 13, 14, 16, 10,  8,  9, 13, 15, 10, 10,  4, 13,
         0,  4,  6, 13,  6, 19, 17,  6,  1, 15, 10,  9,  7, 17, 17,  5,  0,  9,
         6, 14, 16, 18, 17,  5,  3,  0,  9, 18,  7,  5, 11,  7, 11,  2, 12,  0,
        13,  5, 18,  2, 18,  5, 19,  3,  7,  8, 16,  1,  1,  4, 10,  6,  7, 11,
        17, 19, 14, 10,  3,  2,  7,  4,  0,  0, 19, 13, 17, 14,  3, 13, 13,  8,
         0, 15,  3,  0, 14,  0,  2,  3,  9,  9, 16,  9,  2,  4, 17,  4,  2,  2,
        13,  5,  8,  9, 10,  6, 10,  7, 13, 15,  7,  0, 14,  1,  5, 10, 13, 11,
         5,  7,  3, 15, 12, 19, 15, 19, 10,  9, 11,  3, 19, 14,  4, 10,  1, 10,
         7,  2,  6,  9, 16, 11, 15, 12, 11,  3, 19,  3, 10, 12, 19,  8, 

In [61]:
batch_size = 256 # gives the best tradeoff between speed (~10 sec per epoch on T4 GPU) and number of weight updates (32 steps per 10 sec)
dt = 62_500/4
transform = transforms.Compose(
            [
                transforms.CropTime(max=1e6), # all timestamp units in microseconds in Tonic
                transforms.Downsample(spatial_factor=512.0/700.0),
                transforms.ToFrame(
                    sensor_size=(512,1,1),
                    time_window=dt,
                    include_incomplete=True,
                ),
            ]
        )
trainset=datasets.SHD('data', transform=transform)
testset=datasets.SHD('data', transform=transform, train=False)
shd_trainset = DiskCachedDataset(trainset, cache_path='./cache/shd/train/512_'+str(dt)+'/')
shd_testset = DiskCachedDataset(testset, cache_path='./cache/shd/test/512_'+str(dt)+'/')
trainloader = DataLoader(shd_trainset, batch_size=batch_size, collate_fn=tonic.collation.PadTensors(batch_first=False), shuffle=True)
testloader = DataLoader(shd_testset, batch_size=batch_size, collate_fn=tonic.collation.PadTensors(batch_first=False), shuffle=True)

for data, labels in iter(trainloader):
  print(data.size(), labels.size(), labels)
  break

learning_rate = 0.001
num_epochs = 150

# Model, loss function, and optimizer
model = FCN_3(neurons_in=512, neurons_h1=256, neurons_h2=128, neurons_h3=64, bn=False).to(device)
criterion = SF.ce_rate_loss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = lr_scheduler.LinearLR(optimizer, start_factor=1.0, end_factor=0.1, total_iters=100)

t = time.process_time()
train(model, criterion, optimizer, trainloader, device, num_epochs,    testloader, scheduler=scheduler)
print(time.process_time() - t)
accuracy(model, testloader, device)
accuracy_2class(model, testloader, device)

torch.Size([64, 256, 1, 512]) torch.Size([256]) tensor([16, 13,  2, 16, 11, 17, 11,  0,  1,  0, 17,  9,  3,  2,  5, 18,  8,  8,
        12, 13,  5, 18,  5,  1, 13,  8,  0,  2,  8, 16, 17,  4,  8,  1, 17, 16,
         0, 14, 16,  9,  2,  1, 16, 19,  7,  8, 19, 16,  5, 17, 18, 14, 13, 16,
         9, 16,  6, 19,  0,  9, 13,  2, 12, 17, 12,  1,  2,  3,  1, 12, 18, 12,
         2, 18,  8, 10, 11,  3,  9, 14, 18, 13, 15,  3,  2, 19,  4,  3, 10,  2,
         3, 13, 10,  6, 18, 18,  5, 16, 19,  4, 19, 10,  3, 16, 17,  7, 10,  5,
        11,  2,  5,  1, 14,  2, 11, 14,  1,  6, 15, 18, 19, 13,  4,  3, 18, 16,
         4, 11,  7,  7, 14,  4, 10, 14, 15,  6,  0,  5,  1,  0,  7,  6, 19,  5,
        11,  3,  3, 16, 19, 17,  6,  0,  9, 19, 10,  5, 16, 18,  2,  0, 16,  4,
        10,  6,  5,  6, 11, 15, 10,  6,  9,  0, 10,  9, 11, 13,  7, 12,  2,  5,
        19, 11, 10, 16,  0, 11,  1, 12, 14, 13,  7,  0, 14, 17,  6, 18, 14, 10,
         3,  7,  2,  3,  9, 12, 14, 12, 15, 13,  8,  7,  3,  7, 18, 11, 

In [62]:
t = time.process_time()
train(model, criterion, optimizer, trainloader, device, num_epochs=150,    testloader=testloader, scheduler=scheduler)
print(time.process_time() - t)
accuracy(model, testloader, device)
accuracy_2class(model, testloader, device)

Epoch [1/150], Step [1/32], Loss: 2.2848
Accuracy of the network on test set: 73.14% (73.14%)
Epoch [2/150], Step [1/32], Loss: 2.2792
Epoch [3/150], Step [1/32], Loss: 2.2850
Epoch [4/150], Step [1/32], Loss: 2.2729
Epoch [5/150], Step [1/32], Loss: 2.2957
Epoch [6/150], Step [1/32], Loss: 2.2897
Epoch [7/150], Step [1/32], Loss: 2.2901
Epoch [8/150], Step [1/32], Loss: 2.2650
Epoch [9/150], Step [1/32], Loss: 2.2754
Epoch [10/150], Step [1/32], Loss: 2.2867
Epoch [11/150], Step [1/32], Loss: 2.2778
Accuracy of the network on test set: 73.45% (73.45%)
Epoch [12/150], Step [1/32], Loss: 2.2870
Epoch [13/150], Step [1/32], Loss: 2.2849
Epoch [14/150], Step [1/32], Loss: 2.2751
Epoch [15/150], Step [1/32], Loss: 2.2959
Epoch [16/150], Step [1/32], Loss: 2.2709
Epoch [17/150], Step [1/32], Loss: 2.2769
Epoch [18/150], Step [1/32], Loss: 2.2815
Epoch [19/150], Step [1/32], Loss: 2.2752
Epoch [20/150], Step [1/32], Loss: 2.2796
Epoch [21/150], Step [1/32], Loss: 2.2690
Accuracy of the netwo

75% accuracy

* 512x256 = 128K
* 256x128 = 32K
* 128x64  = 8K
*  64x20  = 1.3K

* Total = 170K params

64 timesteps

Training time = ~5000sec = 1.5 hour

In [ ]:

        10.22
        spike_grad = surrogate.fast_sigmoid(slope)

        self.lif2 = snn.Leaky(beta, threshold=self.thr2, reset_mechanism=self.reset_mechanism, spike_grad=spike_grad)

In [ ]:
# ----------------------------------------------------------------------------------------------------

In [ ]:

class CNN(nn.Module):
    def __init__(self, seq_len, neurons_in=512, features=16, kernel_size=16, neurons_h1=256, dropout=0.0, beta=0.95):
        super(FCN_1, self).__init__()
        self.cnn1 = nn.Conv1d(1, features, kernel_size)
        self.fc2 = nn.Linear(features*seq_len, neurons_h1)
        self.fc_ = nn.Linear(neurons_h1, 20)
        self.lif1 = snn.Leaky(beta=beta)
        self.lif_ = snn.Leaky(beta=beta)
        if dropout > 0:
          self.drop1 = nn.Dropout(p=dropout)
        if bn:
          self.bn1 = nn.BatchNorm1d(neurons_h1)

    def forward(self, x):
      spk_rec = []
      mem_rec = []

      mem1 = self.lif1.init_leaky()
      mem_ = self.lif_.init_leaky()

      N = x.size(1)
      X = x.size(3)

      print("in", x.size())
      x = permute(x, (1, 3, 2, 0))              # [T, N, 1, C] -> [N, X, 1, T]
      print("permute(1,3,2,0)", x.size())
      x = cnn1(x)                               # [NC,1T] -cnn-> [NC,KT]
      print("cnn1", x.size())
      x = reshpe(x, (-1, N, 1, X))              # [NX,KT] -> [K*T, N, 1, X]
      print("reshape(-1,N,1,X)", x.size())

      for step in range(x.size(0)):
        cur1 = self.fc1(x[step].squeeze(1))
        if hasattr(self, 'drop1'):
          cur1 = self.drop1(cur1)
        if hasattr(self, 'bn1'):
          cur1 = self.bn1(cur1)
        spk1, mem1 = self.lif1(cur1, mem1)

        cur_ = self.fc_(spk1)
        spk_, mem_ = self.lif_(cur_, mem_)
        spk_rec.append(spk_)
        mem_rec.append(mem_)
      return torch.stack(spk_rec), torch.stack(mem_rec)


class CNN(nn.Module):
    def __init__(self, neurons_in=512, features=16, kernel_size=16, neurons_h1=256, dropout=0.0, beta=0.95):
        super(FCN_1, self).__init__()
        self.cnn1 = nn.Conv1d(1, features, kernel_size)
        self.fc2 = nn.Linear(features, neurons_h1)
        self.fc_ = nn.Linear(neurons_h1, 20)
        self.lif1 = snn.Leaky(beta=beta)
        self.lif_ = snn.Leaky(beta=beta)
        if dropout > 0:
          self.drop1 = nn.Dropout(p=dropout)
        if bn:
          self.bn1 = nn.BatchNorm1d(neurons_h1)

    def forward(self, x):
      spk_rec = []
      mem_rec = []

      mem1 = self.lif1.init_leaky()
      mem_ = self.lif_.init_leaky()

      N = x.size(1)
      X = x.size(3)

      print("in", x.size())
      x = permute(x, (1, 3, 2, 0))              # [T, N, 1, C] -> [N, X, 1, T]
      print("permute(1,3,2,0)", x.size())
      x = cnn1(x)                               # [NC,1T] -cnn-> [NC,KT]
      print("cnn1", x.size())
      x = reshpe(x, (-1, N, 1, X))              # [NX,KT] -> [K*T, N, 1, X]
      print("reshape(-1,N,1,X)", x.size())

      for step in range(x.size(0)):
        cur1 = self.fc1(x[step].squeeze(1))
        if hasattr(self, 'drop1'):
          cur1 = self.drop1(cur1)
        if hasattr(self, 'bn1'):
          cur1 = self.bn1(cur1)
        spk1, mem1 = self.lif1(cur1, mem1)

        cur_ = self.fc_(spk1)
        spk_, mem_ = self.lif_(cur_, mem_)
        spk_rec.append(spk_)
        mem_rec.append(mem_)
      return torch.stack(spk_rec), torch.stack(mem_rec)

In [ ]:
# ----------------------------------------------------------------------------------------------------

In [ ]:
from tonic import datasets, transforms

shd_timestep = 1e-6
shd_channels = 700
net_channels = 64
net_dt = 10e-3
sample_T = 16
batch_size = 4096

class ToRaster():
    def __init__(self, encoding_dim, sample_T = 100):
        self.encoding_dim = encoding_dim
        self.sample_T = sample_T
    def __call__(self, events):
        # tensor has dimensions (time_steps, encoding_dim)
        tensor = np.zeros((events["t"].max()+1, self.encoding_dim), dtype=int)
        np.add.at(tensor, (events["t"], events["x"]), 1)
        return tensor[:self.sample_T,:]

transform = transforms.Compose([
        transforms.Downsample(
            time_factor=shd_timestep / net_dt,
            spatial_factor=net_channels / shd_channels
            ),
        ToRaster(net_channels, sample_T = sample_T),
        torch.Tensor,
        ])

trainset=datasets.SHD('data', transform=transform)
testset=datasets.SHD('data', transform=transform, train=False)
shd_trainset = DiskCachedDataset(trainset, cache_path=f'./cache/shd/train/raster_{sample_T}_{net_dt}_{net_channels}')
shd_testset = DiskCachedDataset(testset, cache_path=f'./cache/shd/test/raster_{sample_T}_{net_dt}_{net_channels}')
trainloader = DataLoader(shd_trainset, batch_size=batch_size, collate_fn=tonic.collation.PadTensors(batch_first=False), shuffle=True)
testloader = DataLoader(shd_testset, batch_size=batch_size, collate_fn=tonic.collation.PadTensors(batch_first=False), shuffle=True)

for data, labels in iter(trainloader):
  print(data.size(), labels.size(), labels)
  print(torch.min(data), torch.max(data), torch.mean(data), torch.var(data))
  break

In [ ]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    total_loss = 0

    for events, labels in testloader:
        events, labels = events.to(device), labels.to(device)
        output, _ = model(torch.Tensor(events).float())

        print (output.size())
        print (output)

        sum = torch.cumsum(output, dim=1)

        predicted = torch.argmax(sum[:,-1,:], 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    accuracy_test = (correct/total)*100

print(f"Test Accuracy: {accuracy_test:.3f}%")